In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import awkward as ak

import matplotlib.pylab as plt
import numpy as np

import time

import pandas as pd
import seaborn as sns

import hist
from hist import Hist

import babar_analysis_tools as bat
import myPIDselector

import os 

from analysis_variables import *


# Read in the data

In [ ]:
BNC_tag = ""
BNC_bool = False

# BNC
#BNC_tag = "_BNC"
#BNC_bool = True

#####################################################################
# Where are we running this?
#####################################################################
## Bellis computer
topdir= "/home/bellis/babar_data/bnv_plambda"

if BNC_bool:
    topdir= "/home/bellis/babar_data/bnv_plambda_bnc"

## My laptop
#topdir= "/Users/josieswann/BaBar_analyses/BNV_pLambda/"
#####################################################################


#####################################################################
# Get the BNV data
#####################################################################
#data_sp, data_collision = bat.load_datasets(topdir=topdir, subset='Run1')
#data_sp, data_collision = bat.load_datasets(topdir=topdir, subset='all')


#####################################################################
# Get the BNC data
#####################################################################
#topdir= "/home/bellis/babar_data/bnv_plambda_bnc"
#data_sp, data_collision = bat.load_datasets(topdir=topdir, BNC=BNC_bool, subset='all')
data_sp, data_collision = bat.load_datasets(topdir=topdir, BNC=BNC_bool, subset='Run1')

#####################################################################
# Get the BNC data
#####################################################################
#topdir= "/home/bellis/babar_data/bnv_plambda_bnc"
#data, data_collision = bat.load_datasets(topdir=topdir, BNC=True, subset='all')
#data, data_collision = bat.load_datasets(topdir=topdir, BNC=True, subset='Run1')



# Trying out PID selector cuts

https://babar-wiki.heprc.uvic.ca/bbr_wiki/index.php/Physics/PID/PID_Selector_List

In [ ]:
# Get these maps first
pps = myPIDselector.PIDselector("p")
pips = myPIDselector.PIDselector("pi")

print(pips.selectors)
print()
print(pps.selectors)

In [ ]:
#'''
proton_selectors_org= []
for i in pps.selectors:
  if "KMProton" in i:
      proton_selectors_org.append(i)

pion_selectors_org=[]
for i in pips.selectors:
  if "KMPion" in i:
      pion_selectors_org.append(i)
#'''
### Smaller list of selectors

#proton_selectors= ["SuperLooseKMProtonSelection","LooseKMProtonSelection","VeryTightKMProtonSelection",]
#pion_selectors= ["SuperLooseKMPionMicroSelection","LooseKMPionMicroSelection","VeryTightKMPionMicroSelection",]

for p in proton_selectors_org:
    print(p)

print()

for p in pion_selectors_org:
    print(p)


# Testing it out

In [ ]:
# Get duplicates mask
mask_event_duplicates= bat.get_duplicates_mask(data_sp)

# Let's make a version of the SP data without duplicates
data = data_sp[mask_event_duplicates]


# This runs the above function
# We pass in the SP awkward array
# It returns a boolean mask for each of the 3 final state particles


mask_bool_proton, mask_bool_pion, mask_bool_protonB = bat.PID_masks(data, \
              lamp_selector='SuperTightKMProtonSelection', \
              lampi_selector='SuperTightKMPionMicroSelection', \
              Bp_selector='LooseKMProtonSelection', \
              verbosity=0)

print(mask_bool_proton)
print(mask_bool_pion)
print(mask_bool_protonB)




In [ ]:
# New we can use those masks


# Let's look at just a specific SP mode
spmask = (data['spmode']=='998')

# We will also make a cut on Lambda flight length because we 
# know we will be doing htat
lamfl_mask = (data['Lambda0FlightLen']>1)

# Use all of the masks uncomment to see (comment the other)
mask_pid =      mask_bool_proton & mask_bool_pion & mask_bool_protonB
# Use two of the masks, uncomment to see
#mask_pid =      mask_bool_proton & mask_bool_pion


# Make some plots
# Get some variables to plot
lammass = data['Lambda0_unc_Mass']
mes = data['BpostFitMes']
#'''
# For each plot, we plot the variable with just the SP mode
# and the Lambda0 flight len, then we plot it again with the PID mask asdded

plt.figure(figsize=(12,5));

plt.subplot(1,2,1)
plt.hist(ak.flatten(lammass[spmask & lamfl_mask]),bins=100)
plt.hist(ak.flatten(lammass[spmask & lamfl_mask & mask_pid]),bins=100)

plt.subplot(1,2,2)
plt.hist(ak.flatten(mes[spmask & lamfl_mask]),bins=100, range=(5.2,5.3))
plt.hist(ak.flatten(mes[spmask & lamfl_mask & mask_pid]),bins=100, range=(5.2,5.3))
#''';

# Let's count the number of events surviving the cuts
n_sp_lam_fl = len(ak.flatten(lammass[spmask & lamfl_mask]))
n_sp_lam_fl_PID = len(ak.flatten(lammass[spmask & lamfl_mask & mask_pid]))

print(f"Before PID: {n_sp_lam_fl}")
print(f"After  PID: {n_sp_lam_fl_PID}")

;

# Sample plots to demonstrate counting

In [ ]:
mes_org = data['BpostFitMes']
de_org = data['BpostFitDeltaE']
#print(mes)


i1= "SuperLooseKMProtonSelection"
j1= "SuperLooseKMProtonSelection"
k1= "SuperLooseKMPionMicroSelection"

i2= "VeryTightKMProtonSelection"
j2= "VeryTightKMProtonSelection"
k2= "VeryTightKMPionMicroSelection"


for i,j,k,pid_tag in zip([i1,i2], [j1,j2], [k1,k2], ['loose', 'very_tight']):

        
    mask_bool_proton, mask_bool_pion, mask_bool_protonB = bat.PID_masks(data, \
                  lamp_selector= i, \
                  lampi_selector= k, \
                  Bp_selector= j, \
              verbosity=0)
    
    for mode in ["-999","998","1005"]:
        spmask = (data['spmode']==mode)
        lamfl_mask = (data['Lambda0FlightLen']>1)
        
        mask_pid = mask_bool_proton & mask_bool_pion & mask_bool_protonB  
    
        # Signal area and fit area mask
        mes_masked= mes_org[mask_pid & spmask & lamfl_mask]
        de_masked= de_org[mask_pid & spmask & lamfl_mask]
    
        plt.figure()
        tag = f'PID_studies_{pid_tag}_{mode}{BNC_tag}'
        bat.plot_mes_vs_DeltaE(ak.flatten(mes_masked), ak.flatten(de_masked), draw_signal_region=True, \
                           draw_sidebands=False, \
                           draw_inference_bins=False, \
                           tag=tag, \
                           region_definitions=region_definitions, \
                           bins=100, zoom=False, plot_full=True)
        

# Do the study

In [ ]:
# For testing with a smaller number
#proton_selectors= ["SuperLooseKMProtonSelection","LooseKMProtonSelection", "VeryTightKMProtonSelection",]
#pion_selectors= ["SuperLooseKMPionMicroSelection","LooseKMPionMicroSelection", "VeryTightKMPionMicroSelection",]

# Final studies
proton_selectors = proton_selectors_org
pion_selectors = pion_selectors_org

# Let's make sure we grab the mass here in the same cell, 
# just in case we ran some other cells. 
lammass = data['Lambda0_unc_Mass']

mes = data['BpostFitMes']
de = data['BpostFitDeltaE']
print(mes)

# Define the dictionary first, which will be used to make the daaframe. 
table = {}
table['icomb'] = []
table['SP mode'] = []
table['Lambda proton selector'] = []
table['Lambda pion selector'] = []
table['B proton selector'] = []
table['# org remaining'] = []
table['# PID remaining'] = []
table['pct remaining'] = []
table['# fit area'] = []
table['# fit area PID'] = []
table['# signal area'] = []
table['# signal area PID'] = []
table['# sideband 1 area'] = []
table['# sideband 1 area PID'] = []
table['# sideband 2 area'] = []
table['# sideband 2 area PID'] = []
table['efficiency'] = []


# Let's keep track of how many combinations we run over
icomb = 1

for i in proton_selectors: 
    print(f"new proton A-------  {i}")
    for j in proton_selectors:
        print(f"\tnew proton B -------- {j}")
        for k in pion_selectors:
            mask_bool_proton, mask_bool_pion, mask_bool_protonB = bat.PID_masks(data, \
                          lamp_selector= i, \
                          lampi_selector= k, \
                          Bp_selector= j, \
                      verbosity=0)

            for mode in ["-999","998","1005"]:
                spmask = (data['spmode']==mode)
                lamfl_mask = (data['Lambda0FlightLen']>1)
                
                mask_pid = mask_bool_proton & mask_bool_pion & mask_bool_protonB                

                # Signal area and fit area mask
                mes_masked= mes[mask_pid & spmask & lamfl_mask]
                de_masked= de[mask_pid & spmask & lamfl_mask]

                signal_area_mask = (mes>5.27) & ((de>-.07) & (de<.07)) 
                fit_area_mask  = (mes>5.2) & ((de>-.2) & (de<.2)) 

                sideband1_mask = (mes>5.27) & ((de>-.14) & (de<-.07)) 
                sideband2_mask = (mes>5.27) & ((de< .14) & (de> .07)) 
                
                n_sp_lam_fl = len(ak.flatten(lammass[spmask & lamfl_mask]))
                n_sp_lam_fl_PID = len(ak.flatten(lammass[spmask & lamfl_mask & mask_pid]))

                n_fit_area = len(ak.flatten(mes[spmask & lamfl_mask & fit_area_mask]))
                n_fit_area_PID = len(ak.flatten(mes[spmask & lamfl_mask & fit_area_mask & mask_pid]))

                n_sig_area = len(ak.flatten(mes[spmask & lamfl_mask & signal_area_mask]))
                n_sig_area_PID = len(ak.flatten(mes[spmask & lamfl_mask & signal_area_mask & mask_pid]))

                n_sideband1_area = len(ak.flatten(mes[spmask & lamfl_mask & sideband1_mask]))
                n_sideband1_area_PID = len(ak.flatten(mes[spmask & lamfl_mask & sideband1_mask & mask_pid]))

                n_sideband2_area = len(ak.flatten(mes[spmask & lamfl_mask & sideband2_mask]))
                n_sideband2_area_PID = len(ak.flatten(mes[spmask & lamfl_mask & sideband2_mask & mask_pid]))
                
                tag = 'bkg'

                # Check to make sure there are entries so we don't get divide by 0
                frac = 0
                if n_sp_lam_fl!=0:
                    frac = n_sp_lam_fl_PID/n_sp_lam_fl

                table['icomb'].append(icomb)
                table['SP mode'].append(mode)

                # Full names
                #table['Lambda proton selector'].append(i)
                #table['Lambda pion selector'].append(k)
                #table['B proton selector'].append(j)

                # Shortened names
                table['Lambda proton selector'].append(i.split('KM')[0])
                table['Lambda pion selector'].append(k.split('KM')[0])
                table['B proton selector'].append(j.split('KM')[0])
                
                table['# org remaining'].append(n_sp_lam_fl)
                table['# PID remaining'].append(n_sp_lam_fl_PID)
                table['pct remaining'].append(100*frac)
                table['efficiency'].append(n_sp_lam_fl_PID/n_sp_lam_fl)

                table['# fit area'].append(n_fit_area)
                table['# fit area PID'].append(n_fit_area_PID)
                table['# signal area'].append(n_sig_area)
                table['# signal area PID'].append(n_sig_area_PID)
                table['# sideband 1 area'].append(n_sideband1_area)
                table['# sideband 1 area PID'].append(n_sideband1_area_PID)
                table['# sideband 2 area'].append(n_sideband2_area)
                table['# sideband 2 area PID'].append(n_sideband2_area_PID)

            icomb += 1

In [ ]:
df_cuts = pd.DataFrame.from_dict(table)
df_cuts

In [ ]:
# Look at just a subset of the dataframe
filter = df_cuts['SP mode']=='1005'
df_cuts[filter]

In [ ]:
# Look at just a subset of the dataframe
filter = df_cuts['SP mode']=='998'
df_cuts[filter]

In [ ]:
# Look at just a subset of the dataframe
filter = df_cuts['SP mode']=='-999'
df_cuts[filter]

In [ ]:
df_cuts.plot.scatter(x='icomb', y='efficiency')

plt.ylim(0,1)

In [ ]:
sns.scatterplot(df_cuts,x='icomb', y='efficiency', hue='SP mode')

plt.ylim(0,1)

In [ ]:
filter998 = df_cuts['SP mode']=='998'
filter1005 = df_cuts['SP mode']=='1005'
filter999 = df_cuts['SP mode']=='-999'

ncombos = len(filter998[filter998])

icomb = df_cuts[filter998]['icomb'].values
lamprot = df_cuts[filter998]['Lambda proton selector'].values
lampion = df_cuts[filter998]['Lambda pion selector'].values
Bprot = df_cuts[filter998]['B proton selector'].values

# The factor of 20 comes from the fact that Run1 is only about 5% of the full dataset. 
# The factors of .3/.25 come the overproduction of those modes, relative to the collision data
bkg998 = df_cuts[filter998]['# signal area PID'].values*20*(0.3)
bkg1005 = df_cuts[filter1005]['# signal area PID'].values*20*(0.25)

#bkgcut= bkg998+bkg1005
#print(bkg998)

sigorg = df_cuts[filter999]['# signal area'].values
sigPID = df_cuts[filter999]['# signal area PID'].values

print(len(sigorg))

sig_eff = sigPID/sigorg
#eff= df_cuts["efficiency"].values

# Punzi figure of merit
a = 4
print(len(sig_eff),len(bkg998), len(bkg1005))
fom = sig_eff/np.sqrt(bkg998 + bkg1005 + (a/2))
#if BNC_bool == True:
#    S = sig_eff*30
#    print(S)
#    B = bkg998 + bkg1005
#    fom = S/np.sqrt(S+B)


# Get the combo
idx = fom.tolist().index(max(fom))
icomb_max = icomb[idx]
print(f"idx: {idx}    icomb_max: {icomb_max}")



plt.figure(figsize=(18,9))
plt.subplot(3,1,1)
plt.plot(icomb, fom, 'o', label='Punzi FOM')
plt.gca().axvline(icomb_max, linestyle='--', color='r', label='max FOM')
plt.ylabel('FOM')
plt.xlabel('PID combination')
plt.legend()

plt.subplot(3,1,2)
plt.plot(icomb, sig_eff, 'o', label='Signal efficiency')
plt.ylim(0,1)
plt.gca().axvline(icomb_max, linestyle='--', color='r', label='max FOM')
plt.ylabel('Signal efficiency')
plt.xlabel('PID combination')
plt.legend()

plt.subplot(3,1,3)
plt.plot(icomb, bkg998 + bkg1005, 'o', label='# background MC in signal region')
plt.ylim(0)
plt.gca().axvline(icomb_max, linestyle='--', color='r', label='max FOM')
plt.ylabel('# of bkg in signal region')
plt.xlabel('PID combination')
plt.legend()

plt.tight_layout()
plt.savefig(f'BNV_pLambda_plots/PID_study_Run1{BNC_tag}.png')

#print(eff)
#bkg1005
;



for idx in range(0,ncombos):
    print(f"FOM: {fom[idx]:.4f}   Eff: {sig_eff[idx]:6.4f}   Bkg: {bkg998[idx] + bkg1005[idx]:5.1f}   Lambda proton: {lamprot[idx]:16s}   Lambda pion: {lampion[idx]:16s}   B proton: {Bprot[idx]:16s}")


print()
print("-----------------------")
print()
idx = fom.tolist().index(max(fom))
print(f"idx: {idx}")

print(f"Max fom: {max(fom)}")
print(f"Eff: {sig_eff[idx]}")
print(f"Bkg: {bkg998[idx] + bkg1005[idx]}")
print(f"Lambda proton: {lamprot[idx]}")
print(f"Lambda pion:   {lampion[idx]}")
print(f"B proton:      {Bprot[idx]}")